In [1]:
import json

import pandas as pd
import numpy as np
import plotly.express as px
import tensorflow.keras.layers as L
import tensorflow as tf
from sklearn.model_selection import train_test_split

**READ JSON FILE**

In [2]:
data_dir = '/kaggle/input/stanford-covid-vaccine/'
train = pd.read_json(data_dir + 'train.json', lines=True)
test = pd.read_json(data_dir + 'test.json', lines=True)
sample_df = pd.read_csv(data_dir + 'sample_submission.csv')

In [3]:
train.shape,test.shape

((2400, 19), (3634, 7))

In [4]:
train.head()

,index,id,sequence,structure,predicted_loop_type,signal_to_noise,SN_filter,seq_length,seq_scored,reactivity_error,deg_error_Mg_pH10,deg_error_pH10,deg_error_Mg_50C,deg_error_50C,reactivity,deg_Mg_pH10,deg_pH10,deg_Mg_50C,deg_50C
0,0,id_001f94081,GGAAAAGCUCUAAUAACAGGAGACUAGGACUACGUAUUUCUAGGUA...,.....((((((.......)))).)).((.....((..((((((......,EEEEESSSSSSHHHHHHHSSSSBSSXSSIIIIISSIISSSSSSHHH...,6.894,1,107,68,"[0.1359, 0.20700000000000002, 0.1633, 0.1452, ...","[0.26130000000000003, 0.38420000000000004, 0.1...","[0.2631, 0.28600000000000003, 0.0964, 0.1574, ...","[0.1501, 0.275, 0.0947, 0.18660000000000002, 0...","[0.2167, 0.34750000000000003, 0.188, 0.2124, 0...","[0.3297, 1.5693000000000001, 1.1227, 0.8686, 0...","[0.7556, 2.983, 0.2526, 1.3789, 0.637600000000...","[2.3375, 3.5060000000000002, 0.3008, 1.0108, 0...","[0.35810000000000003, 2.9683, 0.2589, 1.4552, ...","[0.6382, 3.4773, 0.9988, 1.3228, 0.78770000000..."
1,1,id_0049f53ba,GGAAAAAGCGCGCGCGGUUAGCGCGCGCUUUUGCGCGCGCUGUACC...,.....(((((((((((((((((((((((....)))))))))).)))...,EEEEESSSSSSSSSSSSSSSSSSSSSSSHHHHSSSSSSSSSSBSSS...,0.193,0,107,68,"[2.8272, 2.8272, 2.8272, 4.7343, 2.5676, 2.567...","[73705.3985, 73705.3985, 73705.3985, 73705.398...","[10.1986, 9.2418, 5.0933, 5.0933, 5.0933, 5.09...","[16.6174, 13.868, 8.1968, 8.1968, 8.1968, 8.19...","[15.4857, 7.9596, 13.3957, 5.8777, 5.8777, 5.8...","[0.0, 0.0, 0.0, 2.2965, 0.0, 0.0, 0.0, 0.0, 0....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[4.947, 4.4523, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[4.8511, 4.0426, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[7.6692, 0.0, 10.9561, 0.0, 0.0, 0.0, 0.0, 0.0..."
2,2,id_006f36f57,GGAAAGUGCUCAGAUAAGCUAAGCUCGAAUAGCAAUCGAAUAGAAU...,.....((((.((.....((((.(((.....)))..((((......)...,EEEEESSSSISSIIIIISSSSMSSSHHHHHSSSMMSSSSHHHHHHS...,8.800,1,107,68,"[0.0931, 0.13290000000000002, 0.11280000000000...","[0.1365, 0.2237, 0.1812, 0.1333, 0.1148, 0.160...","[0.17020000000000002, 0.178, 0.111, 0.091, 0.0...","[0.1033, 0.1464, 0.1126, 0.09620000000000001, ...","[0.14980000000000002, 0.1761, 0.1517, 0.116700...","[0.44820000000000004, 1.4822, 1.1819, 0.743400...","[0.2504, 1.4021, 0.9804, 0.49670000000000003, ...","[2.243, 2.9361, 1.0553, 0.721, 0.6396000000000...","[0.5163, 1.6823000000000001, 1.0426, 0.7902, 0...","[0.9501000000000001, 1.7974999999999999, 1.499..."
3,3,id_0082d463b,GGAAAAGCGCGCGCGCGCGCGCGAAAAAGCGCGCGCGCGCGCGCGC...,......((((((((((((((((......))))))))))))))))((...,EEEEEESSSSSSSSSSSSSSSSHHHHHHSSSSSSSSSSSSSSSSSS...,0.104,0,107,68,"[3.5229, 6.0748, 3.0374, 3.0374, 3.0374, 3.037...","[73705.3985, 73705.3985, 73705.3985, 73705.398...","[11.8007, 12.7566, 5.7733, 5.7733, 5.7733, 5.7...","[121286.7181, 121286.7182, 121286.7181, 121286...","[15.3995, 8.1124, 7.7824, 7.7824, 7.7824, 7.78...","[0.0, 2.2399, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0....","[0.0, -0.5083, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0...","[3.4248, 6.8128, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[0.0, -0.8365, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0...","[7.6692, -1.3223, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0..."
4,4,id_0087940f4,GGAAAAUAUAUAAUAUAUUAUAUAAAUAUAUUAUAGAAGUAUAAUA...,.....(((((((.((((((((((((.(((((((((....)))))))...,EEEEESSSSSSSBSSSSSSSSSSSSBSSSSSSSSSHHHHSSSSSSS...,0.423,0,107,68,"[1.665, 2.1728, 2.0041, 1.2405, 0.620200000000...","[4.2139, 3.9637000000000002, 3.2467, 2.4716, 1...","[3.0942, 3.015, 2.1212, 2.0552, 0.881500000000...","[2.6717, 2.4818, 1.9919, 2.5484999999999998, 1...","[1.3285, 3.6173, 1.3057, 1.3021, 1.1507, 1.150...","[0.8267, 2.6577, 2.8481, 0.40090000000000003, ...","[2.1058, 3.138, 2.5437000000000003, 1.0932, 0....","[4.7366, 4.6243, 1.2068, 1.1538, 0.0, 0.0, 0.7...","[2.2052, 1.7947000000000002, 0.7457, 3.1233, 0...","[0.0, 5.1198, -0.3551, -0.3518, 0.0, 0.0, 0.0,..."


In [5]:
test.head()

,index,id,sequence,structure,predicted_loop_type,seq_length,seq_scored
0,0,id_00073f8be,GGAAAAGUACGACUUGAGUACGGAAAACGUACCAACUCGAUUAAAA...,......((((((((((.(((((.....))))))))((((((((......,EEEEEESSSSSSSSSSBSSSSSHHHHHSSSSSSSSSSSSSSSSHHH...,107,68
1,1,id_000ae4237,GGAAACGGGUUCCGCGGAUUGCUGCUAAUAAGAGUAAUCUCUAAAU...,.....((((..((((((...(((((.....((((....)))).......,EEEEESSSSIISSSSSSIIISSSSSIIIIISSSSHHHHSSSSIIII...,130,91
2,2,id_00131c573,GGAAAACAAAACGGCCUGGAAGACGAAGGAAUUCGGCGCGAAGGCC...,...........((.(((.(.(..((..((..((((...))))..))...,EEEEEEEEEEESSISSSISISIISSIISSIISSSSHHHSSSSIISS...,107,68
3,3,id_00181fd34,GGAAAGGAUCUCUAUCGAAGGAUAGAGAUCGCUCGCGACGGCACGA...,......((((((((((....))))))))))((((((..((.(((.....,EEEEEESSSSSSSSSSHHHHSSSSSSSSSSSSSSSSIISSISSSHH...,107,68
4,4,id_0020473f7,GGAAACCCGCCCGCGCCCGCCCGCGCUGCUGCCGUGCCUCCUCUCC...,.....(((((((((((((((((((((((((((((((((((((((((...,EEEEESSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSS...,130,91


In [6]:
sample_df.head()

,id_seqpos,reactivity,deg_Mg_pH10,deg_pH10,deg_Mg_50C,deg_50C
0,id_00073f8be_0,0.0,0.0,0.0,0.0,0.0
1,id_00073f8be_1,0.0,0.0,0.0,0.0,0.0
2,id_00073f8be_2,0.0,0.0,0.0,0.0,0.0
3,id_00073f8be_3,0.0,0.0,0.0,0.0,0.0
4,id_00073f8be_4,0.0,0.0,0.0,0.0,0.0


### seq_scored - (68 in Train and Public Test, 91 in Private Test) 
Integer value denoting the number of positions used in scoring with predicted values. 

This should match the length of `reactivity`, `deg_*` and `*_error_* `columns

In [7]:
print("Unique values & no. of occurences for seq_scored in the training dataset:\n",train.seq_scored.value_counts())
print("\nUnique values & no. of occurences for seq_scored in the test dataset:\n",test.seq_scored.value_counts())

Unique values & no. of occurences for seq_scored in the training dataset:
 68    2400
Name: seq_scored, dtype: int64

Unique values & no. of occurences for seq_scored in the test dataset:
 91    3005
68     629
Name: seq_scored, dtype: int64


Observations:

* We have got the exact count as mentioned on the data page of the competition, all the records present in the training dataset has a value of 68 for seq_scored
    
* Test dataset has 2 different values, 68 for public test data and 91 for private test data
    
* No. of records for different values are also in synch with what is mentioned on the data page, for both train and test datasets



Now let's verify the columns which are tightly bound with seq_scored
* reactivity
* deg_* 
* *_error_*

In [8]:
# training dataset
deg_columns = ['reactivity','deg_error_Mg_pH10', 'deg_error_pH10','deg_error_Mg_50C', 'deg_error_50C', 'deg_Mg_pH10','deg_pH10', 'deg_Mg_50C', 'deg_50C']

for col in deg_columns:
    length = []
    for each in range(train.shape[0]):
        length.append(len(train[col].iloc[each]))

    print("Length of different values for " + col + " in training dataset:",set(length))


Length of different values for reactivity in training dataset: {68}
Length of different values for deg_error_Mg_pH10 in training dataset: {68}
Length of different values for deg_error_pH10 in training dataset: {68}
Length of different values for deg_error_Mg_50C in training dataset: {68}
Length of different values for deg_error_50C in training dataset: {68}
Length of different values for deg_Mg_pH10 in training dataset: {68}
Length of different values for deg_pH10 in training dataset: {68}
Length of different values for deg_Mg_50C in training dataset: {68}
Length of different values for deg_50C in training dataset: {68}



Key Observatoions

* What we verified above is that all the columns in question have values of length 68 for all the records in the training dataset.
This is in sync with the information provided by seq_scored field 


### seq_length - (107 in Train and Public Test, 130 in Private Test) 
Integer values, denotes the length of sequence. Note that molecules used for the Private Test will be longer than those in the Train and Public Test data, so the size of this vector will be different.

In [9]:
print("Unique values & there occurences for seq_length in the training dataset:\n",train.seq_length.value_counts())
print("\nUnique values & there occurences for seq_length in the test dataset:\n",test.seq_length.value_counts())

Unique values & there occurences for seq_length in the training dataset:
 107    2400
Name: seq_length, dtype: int64

Unique values & there occurences for seq_length in the test dataset:
 130    3005
107     629
Name: seq_length, dtype: int64


Observations: 

* For each record in the training dataset, sequence lenght is 107
    
* For test dataset, we have a mix of 130 and 107 sequence lengh which actually represents public and private test data

* Data seems to be in synch with what is mentioned on the data page for both training and test dataset


### structure - (1x107 string in Train and Public Test, 130 in Private Test) 
An array of (, ), and . characters that describe whether a base is estimated to be paired or unpaired. Paired bases are denoted by opening and closing parentheses e.g. (....) means that base 0 is paired to base 5, and bases 1-4 are unpaired.

In [10]:
# training dataset
length = []
for each in range(train.shape[0]):
    length.append(len(train.structure.iloc[each]))

print("length of different values for structure in training dataset:",set(length))

# test dataset
length = []
for each in range(test.shape[0]):
    length.append(len(test.structure.iloc[each]))

print("\nlength of different values for structure in test dataset:",set(length))


length of different values for structure in training dataset: {107}

length of different values for structure in test dataset: {130, 107}


In [11]:
# training dataset
length = []
for each in range(train.shape[0]):
    length.append(len(train.predicted_loop_type.iloc[each]))

print("length of different values for predicted_loop_type in training dataset:",set(length))

# test dataset
length = []
for each in range(test.shape[0]):
    length.append(len(test.predicted_loop_type.iloc[each]))

print("\nlength of different values for predicted_loop_type in test dataset:",set(length))


length of different values for predicted_loop_type in training dataset: {107}

length of different values for predicted_loop_type in test dataset: {130, 107}


Observations: 

* We have now verified that predicted loop has a length of 107 for each record in the training dataset
 
* Whereas test dataset has a mix of 130 and 107 lengths which represents private and public test data


# **DATA PREPROCESSING**

In [12]:
train = train.query("signal_to_noise >= 1")
train.shape

(2097, 19)

### Helper Variables & Functions to pre-process the input data

In [13]:
def pandas_list_to_array(df):
    """
    Input: dataframe of shape (x, y), containing list of length l
    Return: np.array of shape (x, l, y)
    """
    
    return np.transpose(
        np.array(df.values.tolist()),
        (0, 2, 1)
    )

In [14]:
def preprocess_inputs(df, token2int, cols=['sequence', 'structure', 'predicted_loop_type']):
    return pandas_list_to_array(
        df[cols].applymap(lambda seq: [token2int[x] for x in seq])
    )

In [15]:
# predictor variables
pred_cols = ['reactivity', 'deg_Mg_pH10', 'deg_Mg_50C', 'deg_pH10', 'deg_50C']

We have following 3 columns with character sequence:
  
* sequence
* structure 
* predicted_loop_type
    
we need to convert them to integers so that we can feed them into the model.                              


In [16]:
# we are using a dictinoary here to map each character with a unique integer
token2int = {x:i for i, x in enumerate('().ACGUBEHIMSX')}

# calling the function defined above to apply the actual character to integer conversion
# train_inputs is the dataframe we are going to use to feed our keras model
train_inputs = preprocess_inputs(train, token2int)

# call the function to reshape the predictor variables to convert into an array which can be fed into keras model
train_labels = pandas_list_to_array(train[pred_cols])

In [17]:
# sets the random seed
tf.random.set_seed(2020)
np.random.seed(2020)

In [18]:
# This is to generate a new set of random values every time
y_true = tf.random.normal((32, 68, 3))
y_pred = tf.random.normal((32, 68, 3))

In [19]:
# function to calculate average across all RMSE values for each column
def MCRMSE(y_true, y_pred):
    colwise_mse = tf.reduce_mean(tf.square(y_true - y_pred), axis=1)
    return tf.reduce_mean(tf.sqrt(colwise_mse), axis=1)

# GRU LAYER

In [20]:
def gru_layer(hidden_dim, dropout):
    return L.Bidirectional(L.GRU(
        hidden_dim, dropout=dropout, return_sequences=True, kernel_initializer='orthogonal'))

In [21]:
def build_model(embed_size, seq_len=107, pred_len=68, dropout=0.5, 
                sp_dropout=0.2, embed_dim=200, hidden_dim=256, n_layers=3):
    inputs = L.Input(shape=(seq_len, 3))
    embed = L.Embedding(input_dim=embed_size, output_dim=embed_dim)(inputs)
    
    reshaped = tf.reshape(
        embed, shape=(-1, embed.shape[1],  embed.shape[2] * embed.shape[3])
    )
    hidden = L.SpatialDropout1D(sp_dropout)(reshaped)
    
    for x in range(n_layers):
        hidden = gru_layer(hidden_dim, dropout)(hidden)
    

    truncated = hidden[:, :pred_len]
    out = L.Dense(5, activation='linear')(truncated)
    
    model = tf.keras.Model(inputs=inputs, outputs=out)
    model.compile(tf.optimizers.Adam(), loss=MCRMSE)
    
    return model

In [22]:
x_train, x_val, y_train, y_val = train_test_split(
    train_inputs, train_labels, test_size=.1, random_state=34, stratify=train.SN_filter)

In [23]:
public_df = test.query("seq_length == 107")
private_df = test.query("seq_length == 130")

public_inputs = preprocess_inputs(public_df, token2int)
private_inputs = preprocess_inputs(private_df, token2int)

In [24]:
model = build_model(embed_size=len(token2int))
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 107, 3)]          0         
_________________________________________________________________
embedding (Embedding)        (None, 107, 3, 200)       2800      
_________________________________________________________________
tf_op_layer_Reshape (TensorF [(None, 107, 600)]        0         
_________________________________________________________________
spatial_dropout1d (SpatialDr (None, 107, 600)          0         
_________________________________________________________________
bidirectional (Bidirectional (None, 107, 512)          1317888   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 107, 512)          1182720   
_________________________________________________________________
bidirectional_2 (Bidirection (None, 107, 512)         

In [25]:
history = model.fit(
    x_train, y_train,
    validation_data=(x_val, y_val),
    batch_size=32,
    epochs=50,
    verbose=2,
    callbacks=[
        tf.keras.callbacks.ReduceLROnPlateau(patience=5),
        tf.keras.callbacks.ModelCheckpoint('model.h5')
    ]
)


Epoch 1/50
59/59 - 6s - loss: 0.4226 - val_loss: 0.3618
Epoch 2/50
59/59 - 4s - loss: 0.3632 - val_loss: 0.3329
Epoch 3/50
59/59 - 4s - loss: 0.3430 - val_loss: 0.3281
Epoch 4/50
59/59 - 4s - loss: 0.3304 - val_loss: 0.3136
Epoch 5/50
59/59 - 4s - loss: 0.3194 - val_loss: 0.3040
Epoch 6/50
59/59 - 4s - loss: 0.3085 - val_loss: 0.2946
Epoch 7/50
59/59 - 4s - loss: 0.2979 - val_loss: 0.2820
Epoch 8/50
59/59 - 4s - loss: 0.2855 - val_loss: 0.2696
Epoch 9/50
59/59 - 4s - loss: 0.2773 - val_loss: 0.2616
Epoch 10/50
59/59 - 4s - loss: 0.2678 - val_loss: 0.2541
Epoch 11/50
59/59 - 4s - loss: 0.2597 - val_loss: 0.2540
Epoch 12/50
59/59 - 4s - loss: 0.2556 - val_loss: 0.2485
Epoch 13/50
59/59 - 4s - loss: 0.2497 - val_loss: 0.2451
Epoch 14/50
59/59 - 4s - loss: 0.2462 - val_loss: 0.2415
Epoch 15/50
59/59 - 4s - loss: 0.2424 - val_loss: 0.2385
Epoch 16/50
59/59 - 4s - loss: 0.2388 - val_loss: 0.2413
Epoch 17/50
59/59 - 4s - loss: 0.2357 - val_loss: 0.2384
Epoch 18/50
59/59 - 4s - loss: 0.2318 - 

# Let's check how our model is doing

In [26]:
fig = px.line(
    history.history, y=['loss', 'val_loss'],
    labels={'index': 'epoch', 'value': 'MCRMSE'}, 
    title='Training History')
fig.show()

# Predictions

In [27]:
# Caveat: The prediction format requires the output to be the same length as the input,
# although it's not the case for the training data.
model_public = build_model(seq_len=107, pred_len=107, embed_size=len(token2int))
model_private = build_model(seq_len=130, pred_len=130, embed_size=len(token2int))

model_public.load_weights('model.h5')
model_private.load_weights('model.h5')

In [28]:
public_preds = model_public.predict(public_inputs)
private_preds = model_private.predict(private_inputs)

# Post-processing and submit

For each sample, we take the predicted tensors of shape (107, 5) or (130, 5), and convert them to the long format (i.e.  629×107,5  or  3005×130,5 ):

In [29]:
preds_ls = []

for df, preds in [(public_df, public_preds), (private_df, private_preds)]:
    for i, uid in enumerate(df.id):
        single_pred = preds[i]

        single_df = pd.DataFrame(single_pred, columns=pred_cols)
        single_df['id_seqpos'] = [f'{uid}_{x}' for x in range(single_df.shape[0])]

        preds_ls.append(single_df)

preds_df = pd.concat(preds_ls)
preds_df.head()

,reactivity,deg_Mg_pH10,deg_Mg_50C,deg_pH10,deg_50C,id_seqpos
0,0.708256,0.659190,0.546342,2.034063,0.757310,id_00073f8be_0
1,2.104239,3.170810,3.394956,4.254514,2.986442,id_00073f8be_1
2,1.610737,0.718315,0.724157,0.650183,0.793168,id_00073f8be_2
3,1.344301,1.275968,1.805824,1.216240,1.918668,id_00073f8be_3
4,0.858349,0.669474,0.924544,0.475049,0.849421,id_00073f8be_4


In [30]:
submission = sample_df[['id_seqpos']].merge(preds_df, on=['id_seqpos'])
submission.to_csv('submission.csv', index=False)